In [1]:
import lxml
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [2]:
from fake_useragent import UserAgent                
ua = UserAgent()
headers = {'User-Agent': ua.random}


In [3]:
url = "https://www.transfermarkt.es/valencia-cf/kader/verein/1049/plus/0/galerie/0?saison_id=2017"

In [4]:
plantilla18 = requests.get(url, headers=headers)


In [5]:
plantilla18 = BeautifulSoup(plantilla18.text, "html.parser")

In [6]:
temp18={"Nombre":[],
        "Posicion":[],
        "Edad":[],
        "Nacionalidad":[],
        "Valor":[]
        }

SACAR EL VALOR DEL JUGADOR EN EL EQUIPO

In [7]:
for valor in plantilla18.find_all("td", class_ = "rechts hauptlink"):
    temp18["Valor"].append(valor.text)

SACAR NOMBRE JUGADOR

In [8]:
nombres = plantilla18.find_all('img', class_='bilderrahmen-fixed lazy lazy')
for nombre in nombres:
    #jugador24 = nombre["title"]
    temp18["Nombre"].append(nombre["title"])


SACAR EDAD JUGADOR

Veo que desde la posicion 1 y cada 4 saco el TD donde esta la edad ya que ese campo no tiene ningún atributo o clase que pueda diferenciar del resto.
Saco la longitud total de el td para saber su longitud y recorrer cada campo de la edad.

In [9]:
edad = plantilla18.find_all("td",class_ = "zentriert")

longitud=(len(edad)-4) #Saco el total y en el range le pongo 97 porque es el último valor de la cadena que queremos sacar.

valores_edad = [edad[i].text.strip() for i in range(1, longitud, 4)]
temp18["Edad"]=valores_edad

SACAR LA POSICION
Como en el anterior no tenemos una etiqueta ni nada que nos indique el campo de la posicion tenemos que sacarlo
a traves de la posicion de la etiqueta.

In [10]:
posiciones = plantilla18.find_all("table",class_ = "inline-table")

longitud_pos = (len(posiciones))
for i in range(longitud_pos):
    #print(posiciones[i].find_all("td")[2].text.strip())
    temp18["Posicion"].append(posiciones[i].find_all("td")[2].text.strip())

SACAR NACIONALIDAD

saco el td con la clase, veo que los que tienen doble nacionalidad ocupan td diferentes por eso le decimos que coja todos los "img" y la primera posicion de cada td.

In [11]:
nacionalidades = plantilla18.find_all("td", class_='zentriert')
for nacion in nacionalidades:
    img = nacion.find_all("img", class_="flaggenrahmen")
    if img:
        #print(img[0].get("title"))
        temp18["Nacionalidad"].append(img[0].get("title"))

In [12]:
temporada18 =  pd.DataFrame(temp18)

In [13]:
temporada18.head(10)

,Nombre,Posicion,Edad,Nacionalidad,Valor
0,Neto,Portero,28,Brasil,"15,00 mill. €"
1,Jaume Doménech,Portero,27,España,"2,50 mill. €"
2,Cristian Rivero,Portero,20,España,250 mil €
3,Gabriel Paulista,Defensa central,27,Brasil,"20,00 mill. €"
4,Jeison Murillo,Defensa central,26,Colombia,"15,00 mill. €"
5,Ezequiel Garay,Defensa central,31,Argentina,"12,00 mill. €"
6,Rúben Vezo,Defensa central,24,Portugal,"6,00 mill. €"
7,Javi Jiménez,Defensa central,21,España,300 mil €
8,José Gayà,Lateral izquierdo,23,España,"35,00 mill. €"
9,Toni Lato,Lateral izquierdo,20,España,"8,00 mill. €"


In [14]:
temporada18.Edad = temporada18.Edad.astype(int)

In [15]:
temporada18.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Nombre        30 non-null     object
 1   Posicion      30 non-null     object
 2   Edad          30 non-null     int32 
 3   Nacionalidad  30 non-null     object
 4   Valor         30 non-null     object
dtypes: int32(1), object(4)
memory usage: 1.2+ KB


In [16]:
temporada18.Valor = temporada18.Valor.str.replace("[^\d,.]", "", regex=True).str.replace(",", ".").str.extract(r'(\d+.\d+|\d+)').astype(float)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\raul_\AppData\Local\Temp\ipykernel_5836\3659295284.py:1: SyntaxWarning: invalid escape sequence '\d'
  temporada18.Valor = temporada18.Valor.str.replace("[^\d,.]", "", regex=True).str.replace(",", ".").str.extract(r'(\d+.\d+|\d+)').astype(float)


Pasamos los valores a millones, sabemos que los valores altos que habian no corresponden al valor en millones ya que sabemos que 
la plantilla del valencia no tiene jugadores tan valiosos.

In [17]:
temporada18.loc[temporada18.Valor >= 100,"Valor"] = temporada18.loc[temporada18.Valor >= 100,"Valor"]/1000


In [18]:
temporada18["Valor millones"] = temporada18.Valor 

In [19]:
temporada18 = temporada18.drop(columns=["Valor"])

In [20]:
temporada18

,Nombre,Posicion,Edad,Nacionalidad,Valor millones
0,Neto,Portero,28,Brasil,15.00
1,Jaume Doménech,Portero,27,España,2.50
2,Cristian Rivero,Portero,20,España,0.25
3,Gabriel Paulista,Defensa central,27,Brasil,20.00
4,Jeison Murillo,Defensa central,26,Colombia,15.00
5,Ezequiel Garay,Defensa central,31,Argentina,12.00
6,Rúben Vezo,Defensa central,24,Portugal,6.00
7,Javi Jiménez,Defensa central,21,España,0.30
8,José Gayà,Lateral izquierdo,23,España,35.00
9,Toni Lato,Lateral izquierdo,20,España,8.00


In [21]:
#temporada22["Valor millones"].fillna(temporada22["Valor millones"].min, inplace=True)
min_valor = temporada18["Valor millones"].min()
temporada18["Valor millones"].fillna(min_valor, inplace=True)

C:\Users\raul_\AppData\Local\Temp\ipykernel_5836\104178961.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  temporada18["Valor millones"].fillna(min_valor, inplace=True)


In [22]:
ruta="D:\DATASCIENCE\Repositorios\ONLINE_DS_THEBRIDGE_RAULSOLER\Proyecto EDA\LIMGOHOME\DatasLimpios\Temporada18_17.csv"

temporada18.to_csv(ruta, index="False")

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\raul_\AppData\Local\Temp\ipykernel_5836\949997348.py:1: SyntaxWarning: invalid escape sequence '\D'
  ruta="D:\DATASCIENCE\Repositorios\ONLINE_DS_THEBRIDGE_RAULSOLER\Proyecto EDA\LIMGOHOME\DatasLimpios\Temporada18_17.csv"
